In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# Load the necessary data files
df = pd.read_pickle("../data/featurized_data/dov_data.pkl")
df["concatenated_fv"] = df["concatenated_fv"] + df["reverb_feat"]
feat = "concatenated_fv" # Contains the full-length feature vector

In [2]:
df

,acorr_gcc,autocorr_feat,avg_channels,concatenated_fv,distance,doa,filename,gcc_feat,geometry,per_wav_feat,position,reverb_feat,room_id,rotation,session,stat_feat,subject_id,utterance,xcorr_gcc_feat
0,"[0.21896133013069627, 0.18147738464176655, 0.1...","[1.170810888587635, 0.19729458540678002, 0.371...",0.0,"[0.0010266973179442556, 0.0010860743991526229,...",5,45,s1_downstairs_wall_trial1,"[[0.0010266973179442556, 0.0010860743991526229...",open,"[2.898223872351639, 0.5084690361864891, 0.1754...",C1,"[1.312000698717102, 3.3154717253963737, 5.8211...",1.0,0.0,1.0,"[2.692582403567252, 9, 3.5, 0.1623737265610477...",s1,0.0,"[0.9753400973665217, 1.060203367844224, 1.1450..."
1,"[0.13633061945438388, 0.18047590274363765, 0.2...","[1.3765460095500481, 0.40700997970998287, 0.66...",0.0,"[0.00033217213181088126, 0.0003289633200949582...",5,45,s1_downstairs_wall_trial1,"[[0.00033217213181088126, 0.000328963320094958...",open,"[2.242452816569435, 0.18554764545761762, 0.082...",C1,"[0.23635352569385942, 0.6581969907442735, 4.16...",1.0,45.0,1.0,"[1.7320508075688772, 6, 3.0, 0.052900963197685...",s1,0.0,"[1.0458019995130599, 1.1252696285955608, 1.204..."
2,"[0.05148196592926978, 0.02129179146140814, 0.0...","[1.2921498939131935, 0.2433098293840883, 0.348...",0.0,"[0.002731714250321672, 0.00167894224990589, 0....",5,45,s1_downstairs_wall_trial1,"[[0.002731714250321672, 0.00167894224990589, 0...",open,"[1.6346395104968627, 0.11162281332497367, 0.06...",C1,"[0.22965804136245258, 0.5863715419541866, 3.30...",1.0,90.0,1.0,"[1.5, 3, 1.5, 0.044703090661928443, 0.14559831...",s1,0.0,"[0.571480098646134, 0.6091162973704437, 0.6467..."
3,"[0.2507488392293453, 0.2416090248152614, 0.232...","[1.2519229193604982, 0.22517438931390643, 0.46...",0.0,"[5.000037934085622e-05, 0.00017944185961024032...",5,45,s1_downstairs_wall_trial1,"[[5.000037934085622e-05, 0.0001794418596102403...",open,"[1.605999645309845, 0.15394761537610838, 0.095...",C1,"[0.29659971303005056, 0.6771677888031853, 3.34...",1.0,135.0,1.0,"[1.118033988749895, 3, 3.5, 0.0509182840699184...",s1,0.0,"[0.6718124829543134, 0.6887140767648816, 0.705..."
4,"[0.04759088158607485, 0.038223249837756136, 0....","[1.3007311894729257, 0.27396903722546995, 0.58...",0.0,"[0.00012045922838400196, 0.0005541125610004313...",5,45,s1_downstairs_wall_trial1,"[[0.00012045922838400196, 0.000554112561000431...",open,"[1.9549931468208768, 0.3010762408803783, 0.154...",C1,"[0.6568399542006554, 1.7366188433138299, 3.704...",1.0,180.0,1.0,"[0.0, 0, 3.0, 0.06327406400785278, 0.200371845...",s1,0.0,"[0.6032403303931156, 0.6220218295541903, 0.640..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11515,"[0.4656700333580375, 0.46706193219870334, 0.46...","[1.1450765724100913, 0.1049038467463107, 0.332...",0.0,"[0.0030174752966527878, 0.002907737471778652, ...",3,0,s10_downstairs_nowall_trial2,"[[0.0030174752966527878, 0.002907737471778652,...",closed,"[1.5932580463085944, 0.08582008281079766, 0.05...",B0,"[0.053972995412428726, 0.12502588516284252, 1....",0.0,135.0,2.0,"[2.0615528128088303, 6, 3.5, 0.027169490006798...",s10,1.0,"[0.8884345153346658, 0.9028137304509679, 0.917..."
11516,"[0.2136169178411365, 0.21504145395010707, 0.21...","[1.671474636417275, 0.27849502535536896, 0.469...",0.0,"[0.002774047991838688, 0.0010297163856925033, ...",3,0,s10_downstairs_nowall_trial2,"[[0.002774047991838688, 0.0010297163856925033,...",closed,"[1.4931902054121273, 0.05474024019757224, 0.03...",B0,"[0.06856907475207379, 0.1715319224631877, 1.09...",0.0,180.0,2.0,"[2.23606797749979, 6, 2.0, 0.04077382474024689...",s10,1.0,"[0.4691572378699978, 0.4782110044422249, 0.487..."
11517,"[0.3666020426899194, 0.3638495961204171, 0.361...","[1.7044068724712864, 0.44076333194971085, 0.83...",0.0,"[0.0031048558448824133, 0.0014269659105413382,...",3,0,s10_downstairs_nowall_trial2,"[[0.0031048558448824133, 0.0014269659105413382...",closed,"[1.6475054444862882, 0.052256821321919944, 0.0...",B0,"[0.0608762217670190

In [76]:
# Train an eight-way classifier for each rotation angle
X_train = df["concatenated_fv"].tolist()
Y_train = df["rotation"].tolist()
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_train = np.nan_to_num(X_train)
Y_train = Y_train.reshape(-1,1)
min_max_scaler = preprocessing.MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)
clf = ExtraTreesClassifier(n_estimators=1000)
clf.fit(X_train,Y_train)

ExtraTreesClassifier(n_estimators=1000)

In [138]:
# Predict the direction of voice from a sample recording

from featurize1 import features_for_mic_group, get_concat_fv

MIC_GROUP = [[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]]
fs = 48000
c_size = None
avg_channels = False

folder = "../data/dov_testdata_0/taokang_3m_zh_s7_segs/"
for j in range(4):
    filename = f"taokang_3m_zh_s7_{j}"
    wavs, features_gcc, features_xcorr_gcc, acorr_gcc, features_per_wav, extended_fv, autocorr_feat, reverb_sl = features_for_mic_group(folder, filename, fs, MIC_GROUP, c_size, avg_channels)
    fv = get_concat_fv(features_gcc, autocorr_feat, features_per_wav, extended_fv, avg_channels)
    test_fv = [fv + reverb_sl]
    X_test = np.array(test_fv)
    X_test = np.nan_to_num(X_test)
    X_test = min_max_scaler.transform(X_test)
    rot_pred = clf.predict(X_test)[0]

    print("Predicted direction of voice is", rot_pred, "degrees")


Predicted direction of voice is 225.0 degrees
Predicted direction of voice is 270.0 degrees
Predicted direction of voice is 225.0 degrees
Predicted direction of voice is 225.0 degrees


In [10]:
X_test

array([[2.58296803e-03, 4.05281780e-02, 1.92082813e-02, 6.89097116e-04,
        6.74097956e-02, 1.10553584e-01, 2.02527942e-02, 4.88197500e-02,
        1.19511472e-01, 2.20163348e-02, 2.64051393e-02, 1.00000000e+00,
        1.35851524e-02, 5.21386254e-03, 8.97156471e-02, 2.79253174e-02,
        3.69338066e-02, 5.40231340e-02, 2.72193640e-02, 8.38862013e-02,
        6.00760866e-03, 1.74913502e-02, 8.19460182e-02, 3.65295123e-03,
        4.79312410e-02, 6.96129766e-01, 5.04007838e-03, 3.98262605e-02,
        8.47082319e-01, 6.48675138e-03, 1.78298561e-03, 5.53675535e-01,
        1.64652561e-02, 2.33773331e-02, 9.06047930e-02, 7.29440041e-03,
        2.04328809e-02, 2.75942982e-01, 1.64747621e-03, 2.39371518e-02,
        6.10251937e-02, 2.76214986e-02, 4.69812994e-02, 1.08513746e-01,
        8.51518137e-03, 6.96423097e-02, 1.02784533e-02, 7.69423327e-02,
        9.50015297e-01, 5.12276300e-03, 5.81626797e-02, 7.94247229e-01,
        5.73360594e-03, 2.22032386e-02, 4.91028773e-01, 6.587756